In [1]:
import os
os.chdir('/mnt/jw01-aruk-home01/projects/ra_challenge/RA_challenge/RA2_alpine_lads/ra_joint_predictions')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras 
import os
import tensorflow as tf
import tensorflow_addons as tfa
from utils.config import Config

import PIL
import PIL.ImageOps
from tensorflow.keras.utils import plot_model
configuration = Config()
from tensorflow.keras.models import load_model

import itertools
import seaborn as sns

AUTOTUNE = tf.data.experimental.AUTOTUNE

from dataset.ops import dataset_ops
import dataset.ops.image_ops as ops
import model.RSNA_model
import model.NIH_model
from dataset import joint_dataset, rsna_joint_dataset

In [9]:
# reload the library
import importlib
import types
import os

def reload_package(package):
    assert(hasattr(package, "__package__"))
    fn = package.__file__
    fn_dir = os.path.dirname(fn) + os.sep
    module_visit = {fn}
    del fn

    def reload_recursive_ex(module):
        importlib.reload(module)

        for module_child in vars(module).values():
            if isinstance(module_child, types.ModuleType):
                fn_child = getattr(module_child, "__file__", None)
                if (fn_child is not None) and fn_child.startswith(fn_dir):
                    if fn_child not in module_visit:
                        # print("reloading:", fn_child, "from", module)
                        module_visit.add(fn_child)
                        reload_recursive_ex(module_child)

    return reload_recursive_ex(package)

reload_package(model.NIH_model)
reload_package(model.RSNA_model)

In [4]:
NIH_model = model.NIH_model.create_complex_joint_multioutput(configuration)

In [5]:
weights="weights/NIH_new_pretrain_model_100"

In [6]:
NIH_model.load_weights(weights)

In [9]:
keras.models.save_model(NIH_model,"weights/NIH_new_pretrain_model_100.h5")

In [13]:
model.NIH_model.create_VGG_multioutput(configuration).summary()

Model: "complex_multiout_NIH"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   (None, 512)          14713536    input_10[0][0]                   
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 512)          262656      vgg16[1][0]                      
__________________________________________________________________________________________________
batch_normalization_43 (BatchNo (None, 512)          2048        dense_11[0][0]                   
_______________________________________________________________________________

In [22]:
keras.applications.densenet.DenseNet121(include_top=False, weights=None,input_shape=[configuration.img_height,configuration.img_width,1], pooling="avg").summary()

Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 1)  0           input_12[0][0]                   
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 3136        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
________________________________________________________________________________________

In [27]:
RSNA_model = keras.Model(NIH_model.input, NIH_model.layers[-4].output)

In [44]:
RSNA_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 1)]     0         
_________________________________________________________________
conv_1_conv_1 (Conv2D)       (None, 224, 224, 32)      320       
_________________________________________________________________
conv_1_batch_1 (BatchNormali (None, 224, 224, 32)      128       
_________________________________________________________________
conv_1_act_1 (Activation)    (None, 224, 224, 32)      0         
_________________________________________________________________
conv_1_conv_2 (Conv2D)       (None, 224, 224, 32)      9248      
_________________________________________________________________
conv_1_batch_2 (BatchNormali (None, 224, 224, 32)      128       
_________________________________________________________________
conv_1_act_2 (Activation)    (None, 224, 224, 32)      0   

In [16]:
RSNA_NEW_MODEL = model.RSNA_model.complex_joint_finetune_model(configuration)

2020-03-05 15:58:01,482;WARNING - Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.


In [19]:
RSNA_NEW_MODEL.load_weights("weights/complex_model_RSNA_pretrain_with_NIH_pretrainafter_model_200")


In [20]:
keras.models.save_model(RSNA_NEW_MODEL,"resources/complex_model_RSNA_pretrain_with_NIH_pretrain_model_200.h5")

In [7]:
for layer in RSNA_NEW_MODEL.layers:
    layer.trainable = True

In [8]:
joint_dataset, joint_val_dataset = rsna_joint_dataset.rsna_joint_dataset(configuration).create_rsna_joints_dataset(val_split = True)

2020-03-02 21:00:49,023;WARNING - Unresolved object in checkpoint: (root).optimizer.iter
2020-03-02 21:00:49,025;WARNING - Unresolved object in checkpoint: (root).optimizer.beta_1
2020-03-02 21:00:49,027;WARNING - Unresolved object in checkpoint: (root).optimizer.beta_2
2020-03-02 21:00:49,028;WARNING - Unresolved object in checkpoint: (root).optimizer.decay
2020-03-02 21:00:49,029;WARNING - Unresolved object in checkpoint: (root).optimizer.learning_rate
2020-03-02 21:00:49,030;WARNING - A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/alpha/guide/checkpoints#loading_mechanics for details.
2020-03-02 21:00:49,075;WARNING - Missing permissions to create directo

In [10]:
def _split_outcomes(dataset, no_joint_types = 13):
    def __split_outcomes(x, y):
        split_y = tf.split(y, [1, 1, no_joint_types], 1)

        return x, (split_y[0], split_y[1], split_y[2])

    return dataset.map(__split_outcomes, num_parallel_calls=AUTOTUNE)